In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import random
import time
import matplotlib.pyplot as plt
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

train_transforms = transforms.Compose([transforms.Grayscale(),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5,), (0.5,))
                               ])

test_transforms = transforms.Compose([transforms.Grayscale(),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5,), (0.5,))
                               ])
trainpath = r"/content/drive/My Drive/DWT/train"
testpath = r"/content/drive/My Drive/DWT/test"
train_data = datasets.ImageFolder(root=trainpath ,transform=train_transforms)

# Create the validating dataset
test_data = datasets.ImageFolder(root=testpath ,transform=test_transforms)

BATCH_SIZE = 200
train_iterator = torch.utils.data.DataLoader(train_data, 
                                             shuffle = True, 
                                             batch_size = BATCH_SIZE)

test_iterator = torch.utils.data.DataLoader(test_data,
                                            batch_size = BATCH_SIZE)

def im_convert(tensor):
    image = tensor.cpu().clone().detach().numpy()
    image = image.transpose(1, 2, 0)
    image = image * np.array((0.5, 0.5, 0.5)) + np.array((0.5, 0.5, 0.5))
    image = image.clip(0, 1)
    return image

dataiter = iter(train_iterator)
images, labels = dataiter.next()
#fig = plt.figure(figsize=(25, 4))

# for idx in np.arange(20):
#   ax = fig.add_subplot(2, 10, idx+1, xticks=[], yticks=[])
#   plt.imshow(im_convert(images[idx]))
#   ax.set_title([labels[idx].item()])

class Classifier(nn.Module):
    
    def __init__(self, D_in, H1, D_out):
        super().__init__()
        self.linear1 = nn.Linear(D_in, H1)
        self.dropout1 = nn.Dropout(0.2)
        self.linear2 = nn.Linear(H1, D_out)
        self.softmax = torch.nn.Softmax(dim=1)
    def forward(self, x):
        x = F.leaky_relu(self.linear1(x)) 
        x = self.dropout1(x)
        x = self.linear2(x)
        x = self.softmax(x)
        return x

model = Classifier(196, 20, 10)
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = criterion.to(device)

def calculate_accuracy(fx, y):
    preds = fx.argmax(1, keepdim=True)
    correct = preds.eq(y.view_as(preds)).sum()
    acc = correct.float()/preds.shape[0]
    return acc

def train(model, iterator, optimizer, criterion,device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for (x, y) in iterator:
        
        x = x.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
        x = x.view(x.shape[0], -1)       
        fx = model(x)
        
        loss = criterion(fx, y)
        
        acc = calculate_accuracy(fx, y)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator, criterion,device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for (x, y) in iterator:

            x = x.to(device)
            y = y.to(device)
            x = x.view(x.shape[0], -1)
            fx = model(x)

            loss = criterion(fx, y)

            acc = calculate_accuracy(fx, y)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

EPOCHS = 15
for epoch in range(EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion,device)
   
    torch.save(model.state_dict(), '/content/drive/My Drive/Code Video/MNISTOrignal20%drop20softmax.pth')
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')

### evaluating the model ###
model.load_state_dict(torch.load('/content/drive/My Drive/Code Video/MNISTOrignal20%drop20softmax.pth'))
test_loss, test_acc = evaluate(model, test_iterator, criterion,device)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

def im_convert(tensor):
  image = tensor.cpu().clone().detach().numpy()
  image = image.transpose(1, 2, 0)
  image = image * np.array((0.5, 0.5, 0.5)) + np.array((0.5, 0.5, 0.5))
  image = image.clip(0, 1)
  return image

dataiter = iter(test_iterator)
images, labels = dataiter.next()
images = images.to(device)
labels = labels.to(device)
output = model(images.view(images.shape[0], -1))
_, preds = torch.max(output, 1)

fig = plt.figure(figsize=(25, 4))
classes = ('0', '1', '2', '3',
           '4', '5', '6', '7', '8', '9')
for idx in np.arange(20):
  ax = fig.add_subplot(2, 10, idx+1, xticks=[], yticks=[])
  plt.imshow(im_convert(images[idx]))
  ax.set_title("{} ({})".format(str(classes[preds[idx].item()]), str(classes[labels[idx].item()])), color=("green" if preds[idx]==labels[idx] else "red"))